<h1> Assignment (Week 4) Capstone Project #SoBerlin </h1><br><br>
<h1> The story </h1>

<h3> Ever been to Berlin? </h3>
<br>
If yes - you might have heard someone saying "omg thats so Berlin" <br>
<br>
It actually somewhat became a meme. However - what does it even mean to be "so Berlin" <br>
Is there any other city in Germany which is "so (like) Berlin"? <br>
<br> The main question here is: If you need or want to relocate within Germany where would you go?
<br> Many cities in Germany lack a strong reputation or suffer from stereotypes regarding their boringness.
<br> Therefore they might not attract students or young talents
<br> In order to make choosing a city easier or to allow for strategic marketing, 
<br> we want to know, which cities really resemble each other in terms of nightlife, rental prices and potential earnings 
<br> and therefore in their attractiveness to young adults 
<br>
<br> Using a cluster analysis, I’d like to make it easier to compare different cities, so that young people get 
<br> an easier impression whether less famous cities (e.g. Essen, Hanover, Nuremburg) in Germany 
<br> could be attractive compared to big established cities such as Berlin or Munich.  
<br>
<br> Hence, I decided to use: 
<br> a) The average Income
<br> b) The average Rental Prices 
<br> c) A foursquare Profile of possible activities for the night 
<br> <br>
To find clusters of German cities exceeding 500.000 inhabitants
<br>

In [74]:
# Loading dependencies
from bs4 import BeautifulSoup # Used for wiki-scraping

import numpy as np # library to handle data in a vectorized manner
import requests

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

<h2> Preparation </h2>
<br> Extracting the name of all German Cities with more than 500.000 Inhabitants with their respective Size

In [75]:
# Loading the names of "Groß und Mittelstädte"-Table from Wikipedia to find the 700 Cities with > 20.000 Inhabitants
wiki_url = 'https://de.wikipedia.org/wiki/Liste_der_Gro%C3%9F-_und_Mittelst%C3%A4dte_in_Deutschland'
wiki = requests.get(wiki_url).text
html_soup=BeautifulSoup(wiki,'html.parser')
sov_tables = html_soup.find_all('table', class_="wikitable sortable zebra")
df = pd.read_html(str(sov_tables[1]))
df = df[0]

In [76]:
# Additional Cleaning: Remove the dots, to facilitate a change of data type then Change no of inhabitants to a float type
df['2018'] = df['2018'].str.replace('.','')
df['2018'] = pd.to_numeric(df['2018'], errors='coerce')

In [77]:
# Remove Cities with less then 500.000 Inhabitants and columns which aren't needed
df=df[df['2018'] > 500000]
df.drop(['Rang','1970','1980','1990','2000','2010','Bundesland'],axis=1, inplace=True)

<h3> Done... Extracted all the Names of German Cities with more than 500 000 Inhabitants

In [78]:
# Check new data frame
df.columns = ['Stadt','Size']
df

,Stadt,Size
0,Berlin,3644826
1,Hamburg,1841179
2,München,1471508
3,Köln,1085664
4,Frankfurt am Main,753056
5,Stuttgart,634830
6,Düsseldorf,619294
7,Leipzig,587857
8,Dortmund,587010
9,Essen,583109


<h2> Extract additional non-foursquare data </h2>

In [79]:
import wget

url = 'https://github.com/FKlapp/Coursera_Capstone/raw/master/2016_Einkommen_Top15.csv'
url2 = 'https://github.com/FKlapp/Coursera_Capstone/raw/master/2019_Mietpreise.csv'
wget.download(url, '2016_Einkommen_Top15.csv')
wget.download(url2, '2019_Mietpreise.csv')

'2019_Mietpreise (4).csv'

In [80]:
# Load Income-Table and align way of writing Frankfurt 
Income_Table = pd.read_csv('2016_Einkommen_Top15.csv',sep=';',encoding='latin1')
Income_Table.replace({'Frankfurt': 'Frankfurt am Main'},inplace=True)

In [81]:
# Load Rental Prices
Rental_Prices = pd.read_csv('2019_Mietpreise.csv',sep=';',decimal=",",encoding='latin1')
Rental_Prices

,Stadt,Mietpreis H1 2019 in Euro/m²
0,Berlin,11.6
1,München,18.6
2,Nürnberg,10.0
3,Hannover,8.8
4,Stuttgart,13.0
5,Frankfurt am Main,14.2
6,Hamburg,11.8
7,Leipzig,6.8
8,Düsseldorf,10.6
9,Dresden,7.6


In [82]:
# Loop through DataFrame to retrieve Location-Data for the cities
geolocator = Nominatim(user_agent="ger_explorer")

Locations = []

for city in df['Stadt']:
        geoinfo = geolocator.geocode(city)

        Locations.append(
               {
            'Stadt': city,
            'lat': geoinfo.latitude,
            'long':  geoinfo.longitude,
               }  
           )

Locations = pd.DataFrame(Locations)

In [83]:
# Merge City Data with location data
df = pd.merge(df,Locations, on='Stadt')

<h2> Extracting Foursquare Data

In [84]:
CLIENT_ID = 'TKZUDAD3BGL5SQUTQJ31AF1RO5FNLE1XB0QP23EKN3H0WLIR' # your Foursquare ID
CLIENT_SECRET = 'GGHSHKUOEXMK2E4HHLVZW0THPQODRZKS3LXBU3DMRIJ5HDVP' # your Foursquare Secret
VERSION = '20200518' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TKZUDAD3BGL5SQUTQJ31AF1RO5FNLE1XB0QP23EKN3H0WLIR
CLIENT_SECRET:GGHSHKUOEXMK2E4HHLVZW0THPQODRZKS3LXBU3DMRIJ5HDVP


In [86]:
radius = 5000 # Define radius of interest
LIMIT = 100 # How many venues do you want to check?

In [87]:
def getNearbyVenues(names, latitudes, longitudes, catID, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            catID)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [88]:
Nightlife =  getNearbyVenues(names=df['Stadt'],
                        latitudes=df['lat'],
                        longitudes=df['long'],
                        catID='4d4b7105d754a06376d81259',
                        radius=5000)

Berlin
Hamburg
München
Köln
Frankfurt am Main
Stuttgart
Düsseldorf
Leipzig
Dortmund
Essen
Bremen
Dresden
Hannover
Nürnberg


In [89]:
# one hot encoding
nightlife_onehot = pd.get_dummies(Nightlife[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
nightlife_onehot['Neighbourhood'] = Nightlife['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [nightlife_onehot.columns[-1]] + list(nightlife_onehot.columns[:-1])
nightlife_onehot = nightlife_onehot[fixed_columns]

nightlife_grouped = nightlife_onehot.groupby('Neighbourhood').mean().reset_index()
nightlife_grouped=nightlife_grouped.rename({'Neighbourhood':'Stadt'}, axis=1)
#nightlife_grouped.head()

In [90]:
# Merge Data 
Price_Income = pd.merge(Rental_Prices, Income_Table, on='Stadt')
df_cities = pd.merge(Price_Income,df, on='Stadt')

In [91]:
# Merge/Get Cities with their respective nightlife pattern next to demographic data
df_cities = pd.merge(df_cities,nightlife_grouped, on="Stadt")
df_cities.head()

,Stadt,Mietpreis H1 2019 in Euro/m²,Verfügbares Einkommen in Euro,Size,lat,long,African Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Bistro,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cuban Restaurant,Deli / Bodega,Diner,Dive Bar,Eastern European Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Gastropub,Gay Bar,General Entertainment,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Jazz Club,Karaoke Bar,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Nightclub,Other Nightlife,Piano Bar,Pizza Place,Plaza,Polish Restaurant,Pub,Ramen Restaurant,Restaurant,Rhenisch Restaurant,Rock Club,Sake Bar,Seafood Restaurant,Smoke Shop,Spanish Restaurant,Speakeasy,Sports Bar,Tapas Restaurant,Theater,Theme Restaurant,Trattoria/Osteria,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery
0,Berlin,11.6,19.719,3644826,52.517037,13.388860,0.000000,0.0,0.0,0.0,0.0,0.290000,0.00,0.000000,0.020000,0.050000,0.040000,0.000000,0.000000,0.01,0.030000,0.000000,0.0,0.010000,0.150000,0.000000,0.0,0.0,0.0,0.00,0.0,0.01,0.01,0.000000,0.00,0.00,0.00,0.000000,0.040000,0.010000,0.0,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.0,0.0,0.010000,0.01,0.010000,0.000000,0.01,0.020000,0.0,0.0,0.0,0.000000,0.0,0.00,0.050000,0.0,0.01,0.0,0.0,0.000000,0.040000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.010000,0.010000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.130000,0.00,0.0
1,München,18.6,29.685,1471508,48.137108,11.575382,0.000000,0.0,0.0,0.0,0.0,0.240000,0.05,0.000000,0.000000,0.020000,0.070000,0.000000,0.000000,0.00,0.020000,0.000000,0.0,0.090000,0.160000,0.010000,0.0,0.0,0.0,0.01,0.0,0.00,0.00,0.000000,0.01,0.01,0.01,0.000000,0.030000,0.010000,0.0,0.050000,0.000000,0.01,0.000000,0.000000,0.030000,0.0,0.0,0.000000,0.00,0.020000,0.000000,0.00,0.010000,0.0,0.0,0.0,0.000000,0.0,0.01,0.020000,0.0,0.00,0.0,0.0,0.000000,0.040000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.01,0.010000,0.010000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.030000,0.01,0.0
2,Nürnberg,10.0,21.785,518365,49.453872,11.077298,0.000000,0.0,0.0,0.0,0.0,0.253968,0.00,0.015873,0.000000,0.015873,0.079365,0.000000,0.000000,0.00,0.031746,0.000000,0.0,0.079365,0.079365,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.063492,0.000000,0.0,0.015873,0.015873,0.00,0.047619,0.000000,0.047619,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.00,0.031746,0.0,0.0,0.0,0.015873,0.0,0.00,0.063492,0.0,0.00,0.0,0.0,0.015873,0.047619,0.0,0.031746,0.0,0.000000,0.0,0.0,0.0,0.00,0.015873,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.031746,0.00,0.0
3,Hannover,8.8,20.977,538068,52.374478,9.738553,0.000000,0.0,0.0,0.0,0.0,0.108108,0.00,0.000000,0.027027,0.013514,0.081081,0.013514,0.013514,0.00,0.040541,0.013514,0.0,0.040541,0.108108,0.013514,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.040541,0.000000,0.0,0.013514,0.000000,0.00,0.040541,0.000000,0.013514,0.0,0.0,0.013514,0.00,0.013514,0.000000,0.00,0.040541,0.0,0.0,0.0,0.000000,0.0,0.00,0.108108,0.0,0.00,0.0,0.0,0.000000,0.148649,0.0,0.027027,0.0,0.000000,0.0,0.0,0.0,0.00,0.000000,0.013514,0.0,0.013514,0.0,0.0,0.0,0.000000,0.040541,0.00,0.0
4,Stuttgart,13.0,25.012,634830,48.778449,9.180013,0.011236,0.0,0.0,0.0,0.0,0.224719,0.00,0.000000,0.011236,0.000000,0.078652,0.000000,0.000000,0.00,0.011236,0.000000,0.0,0.044944,0.078652,0.000000,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.011236,0.00,0.00,0.00,0.011236,0.056180,0.011236,0.0,0.044944,0.000000,0.00,0.022472,0.011236,0.022472,0.0,0.0,0.000000,0.00,0.000000,0.011236,0.00,0.067416,0.0,0.0,0.0,0.000000,0.0,0.00,0.101124,0.0,0.00,0.0,0.0,0.000000,0.044944,0.0,0.022472,0.0,0.011236,0.0,0.0,0.0,

<h2> Performing Cluster Analysis

In [92]:
# set number of clusters
kclusters = 5

df_clustered = df_cities.drop('Stadt', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_clustered.drop(['Size','lat','long'],1))


In [93]:
# add clustering labels
df_cities.insert(0, 'Cluster Labels', kmeans.labels_)


<h2> Result Map

In [94]:
# create a map of Germany, where the different cities are highlighted and clusters are assigned

map_clusters = folium.Map(location=[49.8, 12], zoom_start=6)

x = np.arange(df_cities['Cluster Labels'].nunique())
ys = [i + x + (i*x)**2 for i in range(df_cities['Cluster Labels'].nunique())]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_cities['lat'], df_cities['long'], df_cities['Stadt'], df_cities['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)],
        fill=True,
        fill_color=rainbow[int(cluster)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h2> Result tables </h2>

<br> Find the Income ("Einkommen") and rental price ("Mietpreis") for each city per cluster
<br> As well as the most common venues (returned for each cluster; not for the specific city)
<br> For an interpretation of these results please see the report or presentation

In [95]:
# Define Function for summary tables per cluster
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[6:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

<h3> Average statistics per Cluster </h3>
<br> rental prices ("Mietpreis"); Accessible Income ("Verfügbares Einkommen") and other summary statistics (e.g. relative frequency of nightlife spots)

In [96]:
# Show average rental price (Mietpreis); Average Income ("Einkommen") and relative frequency of each nighlife spot per Cluster
df_mean = df_cities.groupby("Cluster Labels", as_index=False).mean()
df_mean.head()

,Cluster Labels,Mietpreis H1 2019 in Euro/m²,Verfügbares Einkommen in Euro,Size,lat,long,African Restaurant,American Restaurant,Asian Restaurant,Australian Restaurant,Bakery,Bar,Bavarian Restaurant,Beach,Beach Bar,Beer Bar,Beer Garden,Bistro,Bowling Alley,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Café,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Cuban Restaurant,Deli / Bodega,Diner,Dive Bar,Eastern European Restaurant,Event Space,Fast Food Restaurant,Food & Drink Shop,Food Court,French Restaurant,Gastropub,Gay Bar,General Entertainment,German Restaurant,Greek Restaurant,Hawaiian Restaurant,Hookah Bar,Hotel,Hotel Bar,Ice Cream Shop,Indie Movie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Jazz Club,Karaoke Bar,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Museum,Music Venue,Nightclub,Other Nightlife,Piano Bar,Pizza Place,Plaza,Polish Restaurant,Pub,Ramen Restaurant,Restaurant,Rhenisch Restaurant,Rock Club,Sake Bar,Seafood Restaurant,Smoke Shop,Spanish Restaurant,Speakeasy,Sports Bar,Tapas Restaurant,Theater,Theme Restaurant,Trattoria/Osteria,Vegetarian / Vegan Restaurant,Whisky Bar,Wine Bar,Wine Shop,Winery
0,0,7.100,18.949250,5.781562e+05,51.340603,10.148493,0.000000,0.003788,0.000000,0.003788,0.003906,0.182113,0.00,0.000000,0.000000,0.009770,0.037607,0.005208,0.000000,0.005983,0.031915,0.010417,0.003788,0.046229,0.052048,0.000000,0.003788,0.002604,0.003906,0.00,0.003788,0.003788,0.003378,0.003788,0.00,0.00,0.00,0.000000,0.057839,0.007812,0.002604,0.024704,0.014979,0.00,0.016281,0.010417,0.009361,0.003906,0.006757,0.005208,0.00,0.003378,0.009770,0.002604,0.026825,0.007576,0.010298,0.000000,0.006757,0.003788,0.003378,0.150079,0.007166,0.00,0.000000,0.000,0.000000,0.099397,0.000,0.035047,0.000000,0.002604,0.000,0.000000,0.000000,0.00,0.003378,0.014086,0.000,0.000000,0.002604,0.003378,0.007166,0.00000,0.015871,0.00,0.003378
1,1,11.800,24.771667,1.031768e+06,51.182538,8.655413,0.003745,0.000000,0.003333,0.000000,0.000000,0.205939,0.00,0.000000,0.016468,0.008028,0.047579,0.003333,0.000000,0.000000,0.049802,0.008028,0.000000,0.049817,0.120583,0.003333,0.004695,0.003333,0.000000,0.00,0.000000,0.003333,0.000000,0.003745,0.00,0.00,0.00,0.017079,0.057506,0.003745,0.000000,0.041038,0.000000,0.00,0.007491,0.003745,0.012185,0.000000,0.000000,0.000000,0.00,0.000000,0.003745,0.000000,0.030500,0.000000,0.000000,0.003333,0.000000,0.000000,0.000000,0.075210,0.000000,0.00,0.000000,0.000,0.000000,0.085873,0.000,0.033547,0.003333,0.003745,0.000,0.003333,0.003333,0.00,0.000000,0.019264,0.000,0.000000,0.000000,0.000000,0.000000,0.00844,0.048455,0.00,0.000000
2,2,18.600,29.685000,1.471508e+06,48.137108,11.575382,0.000000,0.000000,0.000000,0.000000,0.000000,0.240000,0.05,0.000000,0.000000,0.020000,0.070000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.090000,0.160000,0.010000,0.000000,0.000000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.01,0.01,0.01,0.000000,0.030000,0.010000,0.000000,0.050000,0.000000,0.01,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.020000,0.000000,0.00,0.000000,0.000,0.000000,0.040000,0.000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.01,0.010000,0.010000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.030000,0.01,0.000000
3,3,12.900,20.704500,2.198941e+06,51.313840,11.035476,0.000000,0.000000,0.000000,0.000000,0.000000,0.255000,0.00,0.000000,0.010000,0.030000,0.035000,0.000000,0.000000,0.005000,0.030000,0.010000,0.000000,0.015000,0.115000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.005000,0.000000,0.00,0.00,0.00,0.005000,0.055000,0.010000,0.000000,0.015000,0.000000,0.00,0.005000,0.000000,0.030000,0.000000,0.000000,0.010000,0.01,0.010000,0.000000,0.010000,0.035000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.055000,0.000000,0.01,0.000000,0.005,0.000000,0.045000,0.005,0.025000,0.0

<h2> Inspection of all clusters focussing on the 10 most common nightlife venues

In [97]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Cluster Labels']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Cluster Labels'] = df_mean['Cluster Labels']

for ind in np.arange(df_mean.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_mean.iloc[ind, :], num_top_venues)

In [98]:
df_ov = pd.merge(df_cities[['Stadt','Cluster Labels','Size','Verfügbares Einkommen in Euro','Mietpreis H1 2019 in Euro/m²']], neighbourhoods_venues_sorted, on='Cluster Labels')

<h3> <br> Cluster 1 The down to earth-Cities

In [99]:
df_ov.loc[df_ov['Cluster Labels'] == 0]

,Stadt,Cluster Labels,Size,Verfügbares Einkommen in Euro,Mietpreis H1 2019 in Euro/m²,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Leipzig,0,587857,17.770,6.8,Bar,Nightclub,Pub,Gastropub,Cocktail Bar,Café,Beer Garden,Restaurant,Brewery,Lounge
11,Dresden,0,554649,18.922,7.6,Bar,Nightclub,Pub,Gastropub,Cocktail Bar,Café,Beer Garden,Restaurant,Brewery,Lounge
12,Dortmund,0,587010,18.946,7.0,Bar,Nightclub,Pub,Gastropub,Cocktail Bar,Café,Beer Garden,Restaurant,Brewery,Lounge
13,Essen,0,583109,20.159,7.0,Bar,Nightclub,Pub,Gastropub,Cocktail Bar,Café,Beer Garden,Restaurant,Brewery,Lounge


<h3> Cluster 2 The Fancy ones

In [100]:
df_ov.loc[df_ov['Cluster Labels'] == 1]

,Stadt,Cluster Labels,Size,Verfügbares Einkommen in Euro,Mietpreis H1 2019 in Euro/m²,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Stuttgart,1,634830,25.012,13.0,Bar,Cocktail Bar,Pub,Nightclub,Gastropub,Café,Brewery,Wine Bar,Beer Garden,German Restaurant
8,Hamburg,1,1841179,24.421,11.8,Bar,Cocktail Bar,Pub,Nightclub,Gastropub,Café,Brewery,Wine Bar,Beer Garden,German Restaurant
9,Düsseldorf,1,619294,24.882,10.6,Bar,Cocktail Bar,Pub,Nightclub,Gastropub,Café,Brewery,Wine Bar,Beer Garden,German Restaurant


<h3> Cluster 3 Well... Munich

In [101]:
df_ov.loc[df_ov['Cluster Labels'] == 2]

,Stadt,Cluster Labels,Size,Verfügbares Einkommen in Euro,Mietpreis H1 2019 in Euro/m²,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,München,2,1471508,29.685,18.6,Bar,Cocktail Bar,Café,Beer Garden,Bavarian Restaurant,German Restaurant,Pub,Hotel Bar,Wine Bar,Gastropub


<h3> Cluster 4 The Cosmopolitans

In [102]:
df_ov.loc[df_ov['Cluster Labels'] == 3]

,Stadt,Cluster Labels,Size,Verfügbares Einkommen in Euro,Mietpreis H1 2019 in Euro/m²,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berlin,3,3644826,19.719,11.6,Bar,Cocktail Bar,Wine Bar,Gastropub,Nightclub,Pub,Lounge,Beer Garden,Beer Bar,Hotel Bar
1,Frankfurt am Main,3,753056,21.690,14.2,Bar,Cocktail Bar,Wine Bar,Gastropub,Nightclub,Pub,Lounge,Beer Garden,Beer Bar,Hotel Bar


<h3> Cluster 5 The normal ones

In [103]:
df_ov.loc[df_ov['Cluster Labels'] == 4]

,Stadt,Cluster Labels,Size,Verfügbares Einkommen in Euro,Mietpreis H1 2019 in Euro/m²,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Nürnberg,4,518365,21.785,10.0,Bar,Cocktail Bar,Pub,Nightclub,Beer Garden,Brewery,Gastropub,Lounge,Café,Hookah Bar
4,Hannover,4,538068,20.977,8.8,Bar,Cocktail Bar,Pub,Nightclub,Beer Garden,Brewery,Gastropub,Lounge,Café,Hookah Bar
5,Köln,4,1085664,21.608,10.9,Bar,Cocktail Bar,Pub,Nightclub,Beer Garden,Brewery,Gastropub,Lounge,Café,Hookah Bar
6,Bremen,4,569352,21.327,8.2,Bar,Cocktail Bar,Pub,Nightclub,Beer Garden,Brewery,Gastropub,Lounge,Café,Hookah Bar
